In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.mam.*
import jline.solvers.nc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.util.matrix.*

In [2]:
// Model creation
val model = Network("myModel")

// Block 1: Network nodes
val source = Source(model, "Source 1")
val queue1 = Queue(model, "Queue 1", SchedStrategy.PS)
val classSwitch = ClassSwitch(model, "ClassSwitch 1")
val sink = Sink(model, "Sink 1")
val queue2 = Queue(model, "Queue 2", SchedStrategy.PS)

// Block 2: Job classes
val jobclassA = OpenClass(model, "Class A", 0)
val jobclassB = OpenClass(model, "Class B", 0)
val jobclassC = OpenClass(model, "Class C", 0)

In [3]:
// Arrival processes
source.setArrival(jobclassA, Exp.fitMean(0.5))    // Class A arrivals
source.setArrival(jobclassB, Exp.fitMean(1.0))    // Class B arrivals
source.setArrival(jobclassC, Disabled.getInstance())  // Class C - no direct arrivals

In [4]:
// Service processes at Queue 1
queue1.setService(jobclassA, Exp.fitMean(0.2))      // Class A
queue1.setService(jobclassB, Exp.fitMean(0.3))      // Class B
queue1.setService(jobclassC, Exp.fitMean(0.333333)) // Class C

// Service processes at Queue 2
queue2.setService(jobclassA, Exp.fitMean(1.0))   // Class A
queue2.setService(jobclassB, Exp.fitMean(1.0))   // Class B
queue2.setService(jobclassC, Exp.fitMean(0.15))  // Class C - fastest service

In [5]:
// Block 3: Network topology and class switching
// Initialize class switching matrix (identity matrix - no class change at the switch node itself)
val C = classSwitch.initClassSwitchMatrix()
// Set identity matrix for no switching at the ClassSwitch node itself
for (i in 0 until model.numberOfClasses) {
    for (j in 0 until model.numberOfClasses) {
        C.set(i, j, if (i == j) 1.0 else 0.0)
    }
}
classSwitch.setClassSwitchingMatrix(C)

In [6]:
// Initialize routing matrix
val P = model.initRoutingMatrix()

// Routing for Class A
P.set(jobclassA, jobclassA, source, queue1, 1.0)      // Source -> Queue1 (Class A)
P.set(jobclassA, jobclassA, queue1, classSwitch, 1.0) // Queue1 -> ClassSwitch (Class A)
P.set(jobclassA, jobclassC, classSwitch, queue2, 1.0) // ClassSwitch -> Queue2 (A->C switch!)
P.set(jobclassC, jobclassC, queue2, sink, 1.0)        // Queue2 -> Sink (Class C)

// Routing for Class B
P.set(jobclassB, jobclassB, source, queue1, 1.0)      // Source -> Queue1 (Class B)
P.set(jobclassB, jobclassB, queue1, classSwitch, 1.0) // Queue1 -> ClassSwitch (Class B)
P.set(jobclassB, jobclassC, classSwitch, queue2, 1.0) // ClassSwitch -> Queue2 (B->C switch!)
P.set(jobclassC, jobclassC, queue2, sink, 1.0)        // Queue2 -> Sink (Class C)

// Routing for Class C (internal routing, no direct arrivals)
P.set(jobclassC, jobclassC, source, queue1, 1.0)      // Source -> Queue1 (not used)
P.set(jobclassC, jobclassC, queue1, classSwitch, 1.0) // Queue1 -> ClassSwitch (not used)
P.set(jobclassC, jobclassC, classSwitch, queue2, 1.0) // ClassSwitch -> Queue2 (Class C)
P.set(jobclassC, jobclassC, queue2, sink, 1.0)        // Queue2 -> Sink (Class C)

model.link(P)

In [7]:
// Create multiple solvers for comparison
val solvers = arrayOf(
    CTMC(model, "keep", true, "verbose", 1, "cutoff", arrayOf(intArrayOf(1,1,0), intArrayOf(3,3,0), intArrayOf(0,0,3))),
    FLD(model, "keep", true, "verbose", 1),
    MVA(model, "keep", true, "verbose", 1),
    MAM(model, "keep", true, "verbose", 1),
    NC(model, "keep", true, "verbose", 1),
    JMT(model, "keep", true, "verbose", 1, "seed", 23000, "samples", 100000),
    SSA(model, "keep", true, "verbose", 1, "seed", 23000, "samples", 100000)
)


Java Modelling Tools cannot be found. LINE will try to download the latest JMT version (download approx. 50MB).
Download started, please wait - this may take several minutes.
Download completed. JMT.jar now located at: /home/gcasale/Dropbox/code/line-dev.git/jar/target/JMT.jar


In [8]:
// Execute all solvers and collect results
for ((i, solver) in solvers.withIndex()) {
    println("\n=== SOLVER ${i+1}: ${solver.name.replace("Solver", "")} ===")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Solver failed: ${e.message}")
    }
}


=== SOLVER 1: CTMC ===
Station   JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source 1  Class A   0           0           0           0           0           1.81587     
Source 1  Class B   0           0           0           0           0           0.94121     
Queue 1   Class A   0.88440     0.36317     0.48704     0.32469     1.81587     1.81587     
Queue 1   Class B   0.71089     0.28236     0.75529     0.25176     0.94121     0.94121     
Queue 2   Class C   0.61834     0.41356     0.22427     0.22427     2.75708     2.75708     
--------------------------------------------------------------------------------------------

=== SOLVER 2: FLD ===
Station   JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source 1  Class A   0  